# Build CLIP Embeddings with OpenVINO

This notebook converts CLIP model to OpenVINO format and builds embeddings database for the dataset.

In [1]:
! pip install git+https://github.com/openai/CLIP.git
! pip install openvino faiss-cpu pillow tqdm

  Cloning https://github.com/openai/CLIP.git to c:\users\sanjay nithin\appdata\local\temp\pip-req-build-bqogca4c
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Sanjay Nithin\AppData\Local\Temp\pip-req-build-bqogca4c'

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import numpy as np
import pickle
from pathlib import Path
from PIL import Image
import torch
import clip
from openvino.runtime import Core
import faiss
from tqdm import tqdm

## Configuration

In [3]:
DATASET_PATH = Path(r"e:\Projects\AI Based\RecTrio\datasets\animals\raw-img")
MODEL_DIR = Path(r"e:\Projects\AI Based\RecTrio\V2\models")
VECTOR_DB_DIR = Path(r"e:\Projects\AI Based\RecTrio\V2\vector_db")

# Create directories if they don't exist
MODEL_DIR.mkdir(parents=True, exist_ok=True)
VECTOR_DB_DIR.mkdir(parents=True, exist_ok=True)

CLIP_MODEL_NAME = "ViT-B/32"  # Fast GitHub CLIP model
VISION_MODEL_PATH = MODEL_DIR / "clip_vision_model.xml"
TEXT_MODEL_PATH = MODEL_DIR / "clip_text_model.xml"

# Embedding database files
EMBEDDINGS_FILE = VECTOR_DB_DIR / "embeddings.npy"
METADATA_FILE = VECTOR_DB_DIR / "metadata.pkl"
FAISS_INDEX_FILE = VECTOR_DB_DIR / "faiss_index.bin"

print(f"✓ Model directory: {MODEL_DIR}")
print(f"✓ Vector DB directory: {VECTOR_DB_DIR}")

✓ Model directory: e:\Projects\AI Based\RecTrio\V2\models
✓ Vector DB directory: e:\Projects\AI Based\RecTrio\V2\vector_db


## Load and Convert CLIP Model to OpenVINO

In [4]:
print("Loading CLIP model from GitHub (fast loading)...")
model, preprocess = clip.load(CLIP_MODEL_NAME, device="cpu")
model.eval()
print(f"✓ CLIP {CLIP_MODEL_NAME} loaded successfully")
print(f"Image input size: 224x224")
print(f"Embedding dimension: 512")

Loading CLIP model from GitHub (fast loading)...
✓ CLIP ViT-B/32 loaded successfully
Image input size: 224x224
Embedding dimension: 512


In [5]:
if not VISION_MODEL_PATH.exists():
    print("Converting vision model to OpenVINO...")
    
    # Create dummy input for vision model
    dummy_input = torch.randn(1, 3, 224, 224)
    
    import openvino as ov
    vision_ov_model = ov.convert_model(
        model.visual,
        example_input=dummy_input
    )
    
    ov.save_model(vision_ov_model, VISION_MODEL_PATH)
    print(f"✓ Vision model saved to {VISION_MODEL_PATH}")
else:
    print("✓ Vision model already exists")

Converting vision model to OpenVINO...


OpConversionFailure: Check 'is_conversion_successful' failed at src\frontends\pytorch\src\frontend.cpp:171:
FrontEnd API failed with OpConversionFailure:
Model wasn't fully converted. Failed operations detailed log:
-- prim::GetAttr with a message:
Exception happened during conversion of operation prim::GetAttr with schema (no schema)
KeyboardInterrupt: <EMPTY MESSAGE>

At:
  e:\Projects\AI Based\RecTrio\venv\Lib\site-packages\openvino\frontend\pytorch\utils.py(96): get_value_from_getattr
  e:\Projects\AI Based\RecTrio\venv\Lib\site-packages\openvino\frontend\pytorch\ts_decoder.py(380): try_decode_get_attr
  e:\Projects\AI Based\RecTrio\venv\Lib\site-packages\openvino\frontend\pytorch\ts_decoder.py(288): visit_subgraph
  e:\Projects\AI Based\RecTrio\venv\Lib\site-packages\openvino\frontend\frontend.py(18): convert
  e:\Projects\AI Based\RecTrio\venv\Lib\site-packages\openvino\tools\ovc\moc_frontend\pipeline.py(143): moc_pipeline
  e:\Projects\AI Based\RecTrio\venv\Lib\site-packages\ope

In [ ]:
if not TEXT_MODEL_PATH.exists():
    print("Converting text encoder to OpenVINO...")
    
    # Create dummy input for text model (tokenized text)
    dummy_text = clip.tokenize(["a photo of a cat"])
    
    # We need to create a wrapper for the text encoding part
    class CLIPTextEncoder(torch.nn.Module):
        def __init__(self, clip_model):
            super().__init__()
            self.clip_model = clip_model
        
        def forward(self, text):
            return self.clip_model.encode_text(text)
    
    text_encoder = CLIPTextEncoder(model)
    text_encoder.eval()
    
    import openvino as ov
    text_ov_model = ov.convert_model(
        text_encoder,
        example_input=dummy_text
    )
    
    ov.save_model(text_ov_model, TEXT_MODEL_PATH)
    print(f"✓ Text encoder saved to {TEXT_MODEL_PATH}")
else:
    print("✓ Text encoder already exists")

✓ Text encoder already exists


## Initialize OpenVINO Runtime

In [ ]:
core = Core()

print("Loading vision model for inference...")
vision_compiled_model = core.compile_model(str(VISION_MODEL_PATH), "CPU")
vision_input_layer = vision_compiled_model.input(0)
vision_output_layer = vision_compiled_model.output(0)

print("✓ Vision model loaded on CPU")
print(f"Input shape: {vision_input_layer.partial_shape}")
print(f"Output shape: {vision_output_layer.partial_shape}")

Loading vision model for inference...
✓ Vision model loaded on CPU
Input shape: [?,3,?,?]
Output shape: [?,512]
✓ Vision model loaded on CPU
Input shape: [?,3,?,?]
Output shape: [?,512]


## Collect Dataset Images

In [ ]:
# Check if embeddings already exist
if EMBEDDINGS_FILE.exists() and METADATA_FILE.exists():
    print("✓ Embeddings database found! Loading existing embeddings...")
    
    # Load embeddings
    embeddings = np.load(EMBEDDINGS_FILE)
    
    # Load metadata
    with open(METADATA_FILE, 'rb') as f:
        metadata = pickle.load(f)
    
    valid_image_paths = metadata['image_paths']
    
    print(f"✓ Loaded {len(embeddings)} embeddings with shape {embeddings.shape}")
    print(f"✓ Total images: {metadata['total_images']}")
    print(f"✓ Embedding dimension: {metadata['embedding_dim']}")
    
    # Set flag to skip generation
    skip_generation = True
else:
    print("No existing embeddings found. Will generate new embeddings...")
    skip_generation = False
    
    # Collect all image paths
    image_paths = []
    supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.webp'}

    for category_dir in DATASET_PATH.iterdir():
        if category_dir.is_dir():
            for img_path in category_dir.iterdir():
                if img_path.suffix.lower() in supported_formats:
                    image_paths.append(str(img_path))

    print(f"Found {len(image_paths)} images in dataset")

No existing embeddings found. Will generate new embeddings...
Found 26179 images in dataset


## Generate Image Embeddings

In [ ]:
def get_image_embedding(image_path):
    """Generate embedding for a single image using OpenVINO CLIP vision model"""
    try:
        # Load and preprocess image using CLIP's preprocessing
        image = Image.open(image_path).convert('RGB')
        image_tensor = preprocess(image).unsqueeze(0)
        
        # Run inference with OpenVINO
        pixel_values = image_tensor.numpy()
        result = vision_compiled_model([pixel_values])[vision_output_layer]
        
        # Get the embedding and normalize
        embedding = result[0]
        embedding = embedding / np.linalg.norm(embedding)
        
        return embedding
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [ ]:
if not skip_generation:
    embeddings = []
    valid_image_paths = []

    print("Generating embeddings...")
    for img_path in tqdm(image_paths, desc="Processing images"):
        embedding = get_image_embedding(img_path)
        if embedding is not None:
            embeddings.append(embedding)
            valid_image_paths.append(img_path)

    embeddings = np.array(embeddings).astype('float32')
    print(f"Generated {len(embeddings)} embeddings with shape {embeddings.shape}")
    
    # Save embeddings to file
    np.save(EMBEDDINGS_FILE, embeddings)
    print(f"✓ Embeddings saved to {EMBEDDINGS_FILE}")
    
    # Save metadata
    metadata = {
        'image_paths': valid_image_paths,
        'total_images': len(valid_image_paths),
        'embedding_dim': embeddings.shape[1]
    }
    
    with open(METADATA_FILE, 'wb') as f:
        pickle.dump(metadata, f)
    print(f"✓ Metadata saved to {METADATA_FILE}")
else:
    print("✓ Skipping embedding generation (already loaded from disk)")

Generating embeddings...


Processing images: 100%|██████████| 26179/26179 [23:11<00:00, 18.82it/s] 



Generated 26179 embeddings with shape (26179, 512)
✓ Embeddings saved to e:\Projects\AI Based\RecTrio\V2\vector_db\embeddings.npy
✓ Metadata saved to e:\Projects\AI Based\RecTrio\V2\vector_db\metadata.pkl


## Build FAISS Index

In [ ]:
if not FAISS_INDEX_FILE.exists():
    print("Building FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    
    index.add(embeddings)
    print(f"✓ FAISS index built with {index.ntotal} vectors")
    
    # Save FAISS index
    faiss.write_index(index, str(FAISS_INDEX_FILE))
    print(f"✓ FAISS index saved to {FAISS_INDEX_FILE}")
else:
    print("✓ FAISS index already exists, loading...")
    index = faiss.read_index(str(FAISS_INDEX_FILE))
    print(f"✓ FAISS index loaded with {index.ntotal} vectors")

Building FAISS index...
✓ FAISS index built with 26179 vectors
✓ FAISS index saved to e:\Projects\AI Based\RecTrio\V2\vector_db\faiss_index.bin


## Save Index and Metadata

In [ ]:
print("\n" + "="*60)
print("EMBEDDING DATABASE SUMMARY")
print("="*60)
print(f"Total images indexed: {len(valid_image_paths)}")
print(f"Embedding dimension: {embeddings.shape[1]}")
print(f"FAISS index size: {index.ntotal} vectors")
print(f"\nFiles saved in: {VECTOR_DB_DIR}")
print(f"  - Embeddings: {EMBEDDINGS_FILE.name}")
print(f"  - Metadata: {METADATA_FILE.name}")
print(f"  - FAISS Index: {FAISS_INDEX_FILE.name}")
print("="*60)
print("\n✓ Embedding database ready for inference!")


EMBEDDING DATABASE SUMMARY
Total images indexed: 26179
Embedding dimension: 512
FAISS index size: 26179 vectors

Files saved in: e:\Projects\AI Based\RecTrio\V2\vector_db
  - Embeddings: embeddings.npy
  - Metadata: metadata.pkl
  - FAISS Index: faiss_index.bin

✓ Embedding database ready for inference!
